In [2]:
import os
import sys
import numpy as np
root = os.path.dirname(os.path.abspath(os.curdir))
sys.path.append(root)

import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torch.utils.data import Subset

from predify.utils.training import train_pcoders, eval_pcoders

from networks_2022 import BranchedNetwork
from data.CleanSoundsDataset import CleanSoundsDataset

# Specify Network to train
TODO: This should be converted to a script that accepts arguments for which network to train

In [13]:
from pbranchednetwork_a1 import PBranchedNetwork_A1SeparateHP
PNetClass = PBranchedNetwork_A1SeparateHP
pnet_name = 'a1'

In [3]:
from pbranchednetwork_conv1 import PBranchedNetwork_Conv1SeparateHP
PNetClass = PBranchedNetwork_Conv1SeparateHP
pnet_name = 'conv1'

In [ ]:
from pbranchednetwork_all import PBranchedNetwork_AllSeparateHP
PNetClass = PBranchedNetwork_AllSeparateHP
pnet_name = 'all'

# Parameters

In [12]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Device: {DEVICE}')
BATCH_SIZE = 64
NUM_WORKERS = 2
PIN_MEMORY = True
NUM_EPOCHS = 50

lr = 1E-4
engram_dir = '/mnt/smb/locker/issa-locker/users/Erica/'
checkpoints_dir = f'{engram_dir}hcnn/checkpoints/'
tensorboard_dir = f'{engram_dir}hcnn/tensorboard/'
datafile = f'{engram_dir}seed_542_word_clean_random_order.hdf5'

Device: cuda:0


In [14]:
!nvidia-smi

Sat May 21 10:15:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 455.45.01    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:1D:00.0 Off |                  N/A |
| 27%   25C    P8    13W / 250W |      3MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Load network and optimizer

In [15]:
net = BranchedNetwork()
net.load_state_dict(torch.load(f'{engram_dir}networks_2022_weights.pt'))

/share/issa/users/es3773/temp-hallucnn/src/models/layers.py:78: UserWarning: Inconsistent tf pad calculation in ConvLayer.
  warnings.warn('Inconsistent tf pad calculation in ConvLayer.')
/share/issa/users/es3773/temp-hallucnn/src/models/layers.py:173: UserWarning: Inconsistent tf pad calculation: 0, 1
  warnings.warn(f'Inconsistent tf pad calculation: {pad_left}, {pad_right}')


<All keys matched successfully>

In [16]:
pnet = PNetClass(net, build_graph=True)

In [17]:
pnet.eval()

PBranchedNetwork_A1SeparateHP(
  (backbone): BranchedNetwork(
    (speech_branch): Sequential(
      (conv1): ConvLayer(
        (block): Sequential(
          (0): Conv2d(1, 96, kernel_size=(6, 14), stride=(3, 3), padding=(2, 6))
          (1): ReLU()
        )
      )
      (rnorm1): LRNorm(
        (block): LocalResponseNorm(5, alpha=0.005, beta=0.75, k=1.0)
      )
      (pool1): PoolLayer(
        (block): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (conv2): ConvLayer(
        (block): Sequential(
          (0): Conv2d(96, 256, kernel_size=(5, 5), stride=(2, 2), padding=(1, 2))
          (1): ReLU()
        )
      )
      (rnorm2): LRNorm(
        (block): LocalResponseNorm(5, alpha=0.005, beta=0.75, k=1.0)
      )
      (pool2): PoolLayer(
        (block): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (conv3): ConvLayer(
        (block): Sequential(
          (0): Conv2d(256, 512, kernel_size=(

In [18]:
pnet.to(DEVICE)
optimizer = torch.optim.Adam(
    [{'params':getattr(pnet,f"pcoder{x+1}").pmodule.parameters(), 'lr':lr} for x in range(pnet.number_of_pcoders)],
    weight_decay=5e-4)

# Set up dataset

In [19]:
full_dataset = CleanSoundsDataset(datafile)
n_train = int(len(full_dataset)*0.9)
train_dataset = Subset(full_dataset, np.arange(n_train))
eval_dataset = Subset(full_dataset, np.arange(n_train, len(full_dataset)))
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
    )
eval_loader = DataLoader(
    eval_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
    )

# Set up checkpoints and tensorboards

In [23]:
checkpoint_path = os.path.join(checkpoints_dir, f"{pnet_name}")
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
checkpoint_path = os.path.join(checkpoint_path, pnet_name + '-{epoch}-{type}.pth')

# summarywriter
from torch.utils.tensorboard import SummaryWriter
tensorboard_path = os.path.join(tensorboard_dir, f"{pnet_name}")
if not os.path.exists(tensorboard_path):
    os.makedirs(tensorboard_path)
sumwriter = SummaryWriter(tensorboard_path, filename_suffix=f'')

# Train

In [ ]:
loss_function = torch.nn.MSELoss()
for epoch in range(1, NUM_EPOCHS+1):
    train_pcoders(pnet, optimizer, loss_function, epoch, train_loader, DEVICE, sumwriter)
    eval_pcoders(pnet, loss_function, epoch, eval_loader, DEVICE, sumwriter)

    # save checkpoints every 5 epochs
    if epoch % 5 == 0:
        torch.save(pnet.state_dict(), checkpoint_path.format(epoch=epoch, type='regular'))

/home/es3773/.local/lib/python3.7/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Training Epoch: 1 [64/36450]	Loss: 7492.5127
Training Epoch: 1 [128/36450]	Loss: 8777.5723
Training Epoch: 1 [192/36450]	Loss: 6675.6455
Training Epoch: 1 [256/36450]	Loss: 7542.8076
Training Epoch: 1 [320/36450]	Loss: 7483.8052
Training Epoch: 1 [384/36450]	Loss: 6024.2134
Training Epoch: 1 [448/36450]	Loss: 5951.9707
Training Epoch: 1 [512/36450]	Loss: 6418.0464
Training Epoch: 1 [576/36450]	Loss: 6349.8696
Training Epoch: 1 [640/36450]	Loss: 5611.2168
Training Epoch: 1 [704/36450]	Loss: 5181.7988
Training Epoch: 1 [768/36450]	Loss: 5816.4316
Training Epoch: 1 [832/36450]	Loss: 5455.4824
Training Epoch: 1 [896/36450]	Loss: 5483.5215
Training Epoch: 1 [960/36450]	Loss: 5065.6382
Training Epoch: 1 [1024/36450]	Loss: 4763.9219
Training Epoch: 1 [1088/36450]	Loss: 5128.7939
Training Epoch: 1 [1152/36450]	Loss: 5030.6118
Training Epoch: 1 [1216/36450]	Loss: 4788.2222
Training Epoch: 1 [1280/36450]	Loss: 4350.8174
Training Epoch: 1 [1344/36450]	Loss: 4531.1528
Training Epoch: 1 [1408/36450

Training Epoch: 1 [11264/36450]	Loss: 1576.4164
Training Epoch: 1 [11328/36450]	Loss: 1596.0035
Training Epoch: 1 [11392/36450]	Loss: 1584.1417
Training Epoch: 1 [11456/36450]	Loss: 1662.4871
Training Epoch: 1 [11520/36450]	Loss: 1491.0220
Training Epoch: 1 [11584/36450]	Loss: 1514.0420
Training Epoch: 1 [11648/36450]	Loss: 1767.6681
Training Epoch: 1 [11712/36450]	Loss: 1498.9977
Training Epoch: 1 [11776/36450]	Loss: 1649.4738
Training Epoch: 1 [11840/36450]	Loss: 1563.4266
Training Epoch: 1 [11904/36450]	Loss: 1546.0212
Training Epoch: 1 [11968/36450]	Loss: 1616.8198
Training Epoch: 1 [12032/36450]	Loss: 1614.4237
Training Epoch: 1 [12096/36450]	Loss: 1626.1426
Training Epoch: 1 [12160/36450]	Loss: 1617.1969
Training Epoch: 1 [12224/36450]	Loss: 1474.4928
Training Epoch: 1 [12288/36450]	Loss: 1601.7637
Training Epoch: 1 [12352/36450]	Loss: 1617.1763
Training Epoch: 1 [12416/36450]	Loss: 1511.8093
Training Epoch: 1 [12480/36450]	Loss: 1557.8224
Training Epoch: 1 [12544/36450]	Loss: 16

Training Epoch: 1 [22208/36450]	Loss: 1181.3229
Training Epoch: 1 [22272/36450]	Loss: 1159.5680
Training Epoch: 1 [22336/36450]	Loss: 1178.3459
Training Epoch: 1 [22400/36450]	Loss: 1098.3834
Training Epoch: 1 [22464/36450]	Loss: 1148.3333
Training Epoch: 1 [22528/36450]	Loss: 1193.0094
Training Epoch: 1 [22592/36450]	Loss: 1165.5717
Training Epoch: 1 [22656/36450]	Loss: 1181.2286
Training Epoch: 1 [22720/36450]	Loss: 1144.0870
Training Epoch: 1 [22784/36450]	Loss: 1193.6759
Training Epoch: 1 [22848/36450]	Loss: 1156.1155
Training Epoch: 1 [22912/36450]	Loss: 1249.4781
Training Epoch: 1 [22976/36450]	Loss: 1160.3521
Training Epoch: 1 [23040/36450]	Loss: 1138.6875
Training Epoch: 1 [23104/36450]	Loss: 1252.1041
Training Epoch: 1 [23168/36450]	Loss: 1142.5894
Training Epoch: 1 [23232/36450]	Loss: 1191.4714
Training Epoch: 1 [23296/36450]	Loss: 1169.9191
Training Epoch: 1 [23360/36450]	Loss: 1159.5917
Training Epoch: 1 [23424/36450]	Loss: 1126.0704
Training Epoch: 1 [23488/36450]	Loss: 11

Training Epoch: 1 [33152/36450]	Loss: 1022.9203
Training Epoch: 1 [33216/36450]	Loss: 1000.8578
Training Epoch: 1 [33280/36450]	Loss: 966.3489
Training Epoch: 1 [33344/36450]	Loss: 968.8422
Training Epoch: 1 [33408/36450]	Loss: 1046.9369
Training Epoch: 1 [33472/36450]	Loss: 1016.9899
Training Epoch: 1 [33536/36450]	Loss: 1065.3562
Training Epoch: 1 [33600/36450]	Loss: 996.1122
Training Epoch: 1 [33664/36450]	Loss: 1016.9005
Training Epoch: 1 [33728/36450]	Loss: 1000.7234
Training Epoch: 1 [33792/36450]	Loss: 1003.9410
Training Epoch: 1 [33856/36450]	Loss: 999.4014
Training Epoch: 1 [33920/36450]	Loss: 1003.3289
Training Epoch: 1 [33984/36450]	Loss: 1007.3377
Training Epoch: 1 [34048/36450]	Loss: 975.3292
Training Epoch: 1 [34112/36450]	Loss: 1053.8929
Training Epoch: 1 [34176/36450]	Loss: 995.7677
Training Epoch: 1 [34240/36450]	Loss: 1007.6777
Training Epoch: 1 [34304/36450]	Loss: 979.8196
Training Epoch: 1 [34368/36450]	Loss: 944.8318
Training Epoch: 1 [34432/36450]	Loss: 1063.9951


...

Training Epoch: 19 [32000/36450]	Loss: 459.8091
Training Epoch: 19 [32064/36450]	Loss: 474.8654
Training Epoch: 19 [32128/36450]	Loss: 490.0107
Training Epoch: 19 [32192/36450]	Loss: 476.0242
Training Epoch: 19 [32256/36450]	Loss: 487.5151
Training Epoch: 19 [32320/36450]	Loss: 484.3176
Training Epoch: 19 [32384/36450]	Loss: 459.9362
Training Epoch: 19 [32448/36450]	Loss: 466.7138
Training Epoch: 19 [32512/36450]	Loss: 478.8326
Training Epoch: 19 [32576/36450]	Loss: 498.1133
Training Epoch: 19 [32640/36450]	Loss: 492.8291
Training Epoch: 19 [32704/36450]	Loss: 495.7122
Training Epoch: 19 [32768/36450]	Loss: 489.6266
Training Epoch: 19 [32832/36450]	Loss: 474.6856
Training Epoch: 19 [32896/36450]	Loss: 494.8031
Training Epoch: 19 [32960/36450]	Loss: 455.4863
Training Epoch: 19 [33024/36450]	Loss: 495.5874
Training Epoch: 19 [33088/36450]	Loss: 483.6913
Training Epoch: 19 [33152/36450]	Loss: 502.7628
Training Epoch: 19 [33216/36450]	Loss: 482.5380
Training Epoch: 19 [33280/36450]	Loss: 4

Training Epoch: 20 [6592/36450]	Loss: 496.4079
Training Epoch: 20 [6656/36450]	Loss: 495.4762
Training Epoch: 20 [6720/36450]	Loss: 465.5015
Training Epoch: 20 [6784/36450]	Loss: 507.8959
Training Epoch: 20 [6848/36450]	Loss: 485.1696
Training Epoch: 20 [6912/36450]	Loss: 465.7502
Training Epoch: 20 [6976/36450]	Loss: 467.1737
Training Epoch: 20 [7040/36450]	Loss: 520.8309
Training Epoch: 20 [7104/36450]	Loss: 478.2328
Training Epoch: 20 [7168/36450]	Loss: 470.4709
Training Epoch: 20 [7232/36450]	Loss: 449.1975
Training Epoch: 20 [7296/36450]	Loss: 499.2498
Training Epoch: 20 [7360/36450]	Loss: 451.9823
Training Epoch: 20 [7424/36450]	Loss: 453.1505
Training Epoch: 20 [7488/36450]	Loss: 488.5554
Training Epoch: 20 [7552/36450]	Loss: 491.0925
Training Epoch: 20 [7616/36450]	Loss: 443.0800
Training Epoch: 20 [7680/36450]	Loss: 488.4024
Training Epoch: 20 [7744/36450]	Loss: 487.9336
Training Epoch: 20 [7808/36450]	Loss: 494.9665
Training Epoch: 20 [7872/36450]	Loss: 480.0995
Training Epoc

Training Epoch: 20 [17600/36450]	Loss: 508.5437
Training Epoch: 20 [17664/36450]	Loss: 464.8615
Training Epoch: 20 [17728/36450]	Loss: 499.3744
Training Epoch: 20 [17792/36450]	Loss: 468.4121
Training Epoch: 20 [17856/36450]	Loss: 516.9742
Training Epoch: 20 [17920/36450]	Loss: 448.9382
Training Epoch: 20 [17984/36450]	Loss: 459.2543
Training Epoch: 20 [18048/36450]	Loss: 486.8914
Training Epoch: 20 [18112/36450]	Loss: 502.4834
Training Epoch: 20 [18176/36450]	Loss: 453.4332
Training Epoch: 20 [18240/36450]	Loss: 477.8468
Training Epoch: 20 [18304/36450]	Loss: 478.8470
Training Epoch: 20 [18368/36450]	Loss: 498.2411
Training Epoch: 20 [18432/36450]	Loss: 472.4548
Training Epoch: 20 [18496/36450]	Loss: 465.8032
Training Epoch: 20 [18560/36450]	Loss: 498.4285
Training Epoch: 20 [18624/36450]	Loss: 472.3718
Training Epoch: 20 [18688/36450]	Loss: 471.0480
Training Epoch: 20 [18752/36450]	Loss: 499.2217
Training Epoch: 20 [18816/36450]	Loss: 472.2079
Training Epoch: 20 [18880/36450]	Loss: 4

Training Epoch: 20 [28544/36450]	Loss: 485.8538
Training Epoch: 20 [28608/36450]	Loss: 486.0641
Training Epoch: 20 [28672/36450]	Loss: 470.5573
Training Epoch: 20 [28736/36450]	Loss: 508.7047
Training Epoch: 20 [28800/36450]	Loss: 506.7592
Training Epoch: 20 [28864/36450]	Loss: 474.0175
Training Epoch: 20 [28928/36450]	Loss: 472.5833
Training Epoch: 20 [28992/36450]	Loss: 485.6914
Training Epoch: 20 [29056/36450]	Loss: 458.6581
Training Epoch: 20 [29120/36450]	Loss: 463.9803
Training Epoch: 20 [29184/36450]	Loss: 471.0584
Training Epoch: 20 [29248/36450]	Loss: 449.5945
Training Epoch: 20 [29312/36450]	Loss: 479.8055
Training Epoch: 20 [29376/36450]	Loss: 483.6069
Training Epoch: 20 [29440/36450]	Loss: 492.4164
Training Epoch: 20 [29504/36450]	Loss: 477.0094
Training Epoch: 20 [29568/36450]	Loss: 483.4731
Training Epoch: 20 [29632/36450]	Loss: 453.5063
Training Epoch: 20 [29696/36450]	Loss: 488.0104
Training Epoch: 20 [29760/36450]	Loss: 497.2382
Training Epoch: 20 [29824/36450]	Loss: 4

Training Epoch: 21 [3072/36450]	Loss: 492.6668
Training Epoch: 21 [3136/36450]	Loss: 487.7713
Training Epoch: 21 [3200/36450]	Loss: 454.0619
Training Epoch: 21 [3264/36450]	Loss: 477.6951
Training Epoch: 21 [3328/36450]	Loss: 542.0675
Training Epoch: 21 [3392/36450]	Loss: 474.1008
Training Epoch: 21 [3456/36450]	Loss: 482.0456
Training Epoch: 21 [3520/36450]	Loss: 461.3793
Training Epoch: 21 [3584/36450]	Loss: 467.0254
Training Epoch: 21 [3648/36450]	Loss: 474.1366
Training Epoch: 21 [3712/36450]	Loss: 465.6810
Training Epoch: 21 [3776/36450]	Loss: 495.8764
Training Epoch: 21 [3840/36450]	Loss: 495.4139
Training Epoch: 21 [3904/36450]	Loss: 471.0454
Training Epoch: 21 [3968/36450]	Loss: 487.2742
Training Epoch: 21 [4032/36450]	Loss: 488.9745
Training Epoch: 21 [4096/36450]	Loss: 483.8677
Training Epoch: 21 [4160/36450]	Loss: 446.2561
Training Epoch: 21 [4224/36450]	Loss: 464.4409
Training Epoch: 21 [4288/36450]	Loss: 474.5958
Training Epoch: 21 [4352/36450]	Loss: 505.1618
Training Epoc